# Hash Consultation

First let's ensure we are consulting with the blockchain and redis database:

In [1]:
%ENV
BLOCKCHAIN=1
REDIS=1
TIMESTAMPING=1
LOGGING_LEVEL=20

SET ENV BLOCKCHAIN=1
SET ENV REDIS=1
SET ENV TIMESTAMPING=1
SET ENV LOGGING_LEVEL=20

Now, let's run a Prolog cell to store its information on chain (and in the redis database):

In [3]:
%file: socrates.pl

man(socrates).
mortal(X) :- man(X).

Redis storing feffe35661e85f6b086319d4ec94e8bd@swipluser-socrates-hash
File: socrates.pl
Timestamp: 1644275135685670630
Hash: feffe35661e85f6b086319d4ec94e8bd
Domain: swipluser-socrates-hash
Iroha Response: COMMITTED

After running this cell, copy the redis database information (looking something like `<hash>@swipluser-socrates-hash`) and restart the kernel. This clears the current Prolog factbase. 

---
After restarting the kernel, set the environment variables again:

In [1]:
%ENV
BLOCKCHAIN=1
REDIS=1
TIMESTAMPING=1
LOGGING_LEVEL=20

SET ENV BLOCKCHAIN=1
SET ENV REDIS=1
SET ENV TIMESTAMPING=1
SET ENV LOGGING_LEVEL=20

To prove the kernel has restarted and cleared the factbase, run the following query and see the errors:

In [2]:
?- mortal(socrates).
?- mortal(X).
?- mortal(bob).

ERROR: Caused by: '  mortal(socrates)'. Returned: 'error(existence_error(procedure, /(mortal, 1)), context(/(pyrun, 2), _2260))'.
ERROR: Caused by: '  mortal(X)'. Returned: 'error(existence_error(procedure, /(mortal, 1)), context(/(pyrun, 2), _2362))'.
ERROR: Caused by: '  mortal(bob)'. Returned: 'error(existence_error(procedure, /(mortal, 1)), context(/(pyrun, 2), _2452))'.

If you have lost your hash information from the previous kernel (or want to confirm the information is correct), we can query the domain on the chain instead:

In [3]:
%python
from IrohaUtils import *
import json
custodian = IrohaHashCustodian.Custodian()

hashes = custodian.get_domain_hashes("swipluser-socrates-hash")
for hash in hashes: print(json.dumps(hash, indent=2))

{
  "hash": "10c091ab624de8781df5250486b537ab",
  "height": 6,
  "domain": "swipluser-socrates-hash",
  "creator_id": "swipluser@hashing",
  "time": 1644268194036
}
{
  "hash": "b507a51e9a7c3897ab2c09e58d2cb393",
  "height": 9,
  "domain": "swipluser-socrates-hash",
  "creator_id": "swipluser@hashing",
  "time": 1644268275799
}

We can now use the hash from our previous kernel and load it into our current factbase using `%consult` magic:

In [4]:
%consult
b507a51e9a7c3897ab2c09e58d2cb393@swipluser-socrates-hash

b507a51e9a7c3897ab2c09e58d2cb393@swipluser-socrates-hash
% 1644268273029744676
man(socrates).
mortal(X) :- man(X).
--------------------------------------------------------------------------------
Successfully consulted socrates.pl
--------------------------------------------------------------------------------


And now we can run the same query and get the correct results, demonstrating that the factbase from the above hash has indeed been loaded into this kernel


In [5]:
?- mortal(socrates).
?- mortal(X).
?- mortal(bob).

true.
X = socrates .
false.